In [ ]:
pip install --no-cache-dir -I tensorflow==2.5

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,Model,Sequential
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/Final_resized_traintest15_train19_1.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Final_resized_traintest15_train19_1/36598_left.jpg  
  inflating: Final_resized_traintest15_train19_1/20517_left.jpg  
  inflating: Final_resized_traintest15_train19_1/188219f2d9c6.jpg  
  inflating: Final_resized_traintest15_train19_1/35291_right.jpg  
  inflating: Final_resized_traintest15_train19_1/14690_left.jpg  
  inflating: Final_resized_traintest15_train19_1/41930_right.jpg  
  inflating: Final_resized_traintest15_train19_1/11994_right.jpg  
  inflating: Final_resized_traintest15_train19_1/18656_right.jpg  
  inflating: Final_resized_traintest15_train19_1/8733_left.jpg  
  inflating: Final_resized_traintest15_train19_1/43660_right.jpg  
  inflating: Final_resized_traintest15_train19_1/21414_right.jpg  
  inflating: Final_resized_traintest15_train19_1/34735_left.jpg  
  inflating: Final_resized_traintest15_train19_1/13648_left.jpg  
  inflating: Final_resized_traintest15_train19_1/4442_left.jpg  
  inflating: Final_

In [ ]:
import os
print(len(os.listdir('/content/Final_resized_traintest15_train19_1')))

92364


In [ ]:
df =pd.read_csv("df_res_tvt.csv")

In [ ]:
df.shape

(92364, 4)

In [ ]:
df.head()

,id_code,diagnosis,file_path,file_name
0,000c1434d8d7,2,/content/resized_traintest15_train19/000c1434d...,000c1434d8d7.jpg
1,001639a390f0,4,/content/resized_traintest15_train19/001639a39...,001639a390f0.jpg
2,0024cdab0c1e,1,/content/resized_traintest15_train19/0024cdab0...,0024cdab0c1e.jpg
3,002c21358ce6,0,/content/resized_traintest15_train19/002c21358...,002c21358ce6.jpg
4,005b95c28852,0,/content/resized_traintest15_train19/005b95c28...,005b95c28852.jpg


In [ ]:
df["diagnosis"]=df["diagnosis"].astype(str)

In [ ]:
# sklearn librairies..
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2)
print(df_train.shape, df_test.shape)

(73891, 4) (18473, 4)


In [ ]:
# Model parameters
BATCH_SIZE = 8
EPOCHS = 40
WARMUP_EPOCHS = 2
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 512
WIDTH = 512
CANAL = 3
N_CLASSES = df['diagnosis'].nunique()
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2,horizontal_flip=True)
    
train_generator=train_datagen.flow_from_dataframe(dataframe=df_train,
                                                  directory="/content/Final_resized_traintest15_train19_1",
                                                  x_col="file_name",
                                                  y_col="diagnosis",
                                                  batch_size=BATCH_SIZE,
                                                  class_mode="categorical",
                                                  target_size=(HEIGHT, WIDTH),
                                                  subset='training')

valid_generator=train_datagen.flow_from_dataframe(dataframe=df_test,
                                                  directory="/content/Final_resized_traintest15_train19_1",
                                                  x_col="file_name",
                                                  y_col="diagnosis",
                                                  batch_size=BATCH_SIZE,
                                                  class_mode="categorical",    
                                                  target_size=(HEIGHT, WIDTH),
                                                  subset='validation')

Found 59113 validated image filenames belonging to 5 classes.
Found 3694 validated image filenames belonging to 5 classes.


In [ ]:

def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(weights=None, include_top=False,input_tensor=input_tensor)

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

In [ ]:
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation
from tensorflow.keras import applications
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau


In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print(STEP_SIZE_TRAIN,STEP_SIZE_VALID)

7389 461


In [ ]:
model = create_model(input_shape=(HEIGHT, WIDTH, CANAL), n_out=N_CLASSES)
for layer in model.layers:
  layer.trainable = False
for i in range(-5, 0):
  model.layers[i].trainable = True
  
es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

callback_list = [es, rlrop]
optimizer = optimizers.Adam(lr=LEARNING_RATE)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.compile(
        optimizer=optimizer,
        loss= tf.keras.losses.categorical_crossentropy,
        metrics=['categorical_accuracy']
    )
checkpoint_path = "path/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_weights_only=True,
        period=5)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
# training the model
history_finetunning = model.fit_generator(generator=train_generator,
                                          steps_per_epoch=STEP_SIZE_TRAIN,
                                          validation_data=valid_generator,
                                          validation_steps=STEP_SIZE_VALID,
                                          epochs=EPOCHS,
                                          callbacks=callback_list,
                                          verbose=1).history                                        

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
6090/7389 [=======================>......] - ETA: 2:28:12 - loss: 0.9057 - categorical_accuracy: 0.7273